# Cameron Station Commuter Invoice Creator

The purpose of the code in here is to generate nice and easy invoices for our various commuters throughout a given period of time (usually a calendar month).

Ultimately, this code should record rides by date, sort them into rider-specific data frames, and spit out a nicely formatted string/invoice object (if I can find a Python package for that) that will tell a given rider the dates and times (e.g. Morning or Evening) that they rode with us, plus the total amount they owe us.

**First things first, export the Commuter Google calendar as `calendar_data.ics` so we can ingest it!**

## Ingesting Google Calendar export

OK, now that we have our calender file, let's continue by defining a function that will allow us to pull in icalendar data objects and spit out useful strings.

In [5]:
from icalendar import Calendar, Event
import datetime as dt
import pandas as pd

MORNING_COMMUTE_TIME = dt.time(hour = 8, minute = 0)
EVENING_COMMUTE_TIME = dt.time(hour = 17, minute = 15)

#Rate (in USD) per commuter per ride
GOING_RATE = 3

def record_maker(event_summary, event_datetime):
    """
    Extract info about commuter event and return in a useful format
    
    event_summary: icalendar component summary vText describing the title of the event. Expected
                    to be of the format "Passenger #X: Passenger_Name"
    event_date: icalendar component datetime value. Provides info on both the date of the commute
                and the time (morning vs. evening) that the commute took place.
                
    Returns list of format [Date, Commute Time, Passenger Name]
    """
    
    #[Passenger Num Label, Passenger Name]
    passenger_info = event_summary.split(": ")

    if event_datetime.time() == MORNING_COMMUTE_TIME:
        commute_time = "Morning"        
    elif event_datetime.time() == EVENING_COMMUTE_TIME:
        commute_time = "Evening"
    else:
        commute_time = "ERROR! Event time doesn't match commute windows!"
        
    #Check to make sure it actually found a name to match to
    if len(passenger_info) > 1 and passenger_info[1] != "":
        output = [event_datetime.date().strftime("%x"), commute_time, passenger_info[1].title()]
        return output
    
           
    else:
        return []

In [24]:
#The start and end dates the invoice should cover. The invoice will be INCLUSIVE of these dates
invoice_startdate = dt.date(2018,4,30)
invoice_enddate = dt.date(2018,5,31)

data_dict = {"Date": [],
            "Commute Time": [],
            "Passenger Name": []}

with open('calendar_data.ics','rb') as f:
    gcal = Calendar.from_ical(f.read())
    for component in gcal.walk():
        temp_data = []
        
        #Check to make sure we skip useless calendar data
        if component.get('summary'):
            summary = component.get('summary')
            #print(summary)
            dtstart = component.get('dtstart').dt
            #print(dtstart)
            
            #Check to make sure we're only looking at the period of time we want to invoice
            if dtstart.date() >= invoice_startdate and dtstart.date() <= invoice_enddate:
                temp_data = record_maker(summary, dtstart)
                
                #Make sure we're only recording records that include actual passengers
                if temp_data:
                    data_dict["Date"].append(temp_data[0])
                    data_dict['Commute Time'].append(temp_data[1])
                    data_dict["Passenger Name"].append(temp_data[2])

## Making a Nice Date- and Time-Resolved Data Frame

In [25]:
full_record = pd.DataFrame(data_dict)
full_record.sort_values(["Date", "Passenger Name"], inplace=True)
full_record

,Commute Time,Date,Passenger Name
9,Evening,04/30/18,Courtney
5,Morning,04/30/18,Paul (credit - don't count)
6,Evening,04/30/18,Paul (credit - don't count)
0,Morning,05/07/18,Paul
3,Evening,05/07/18,Paul
8,Evening,05/07/18,Sarah
1,Morning,05/09/18,Paul
4,Evening,05/09/18,Paul
2,Morning,05/10/18,Paul
7,Evening,05/10/18,Paul


## Time to Generate That Invoice

Likely the easiest thing to do here is to output an Excel file that only spits out dates of rides and commute segments (e.g. Morning or Evening) for each rider, making a new sheet for each new rider name. Then I can copy and paste those data sets into my Word-based invoice template and VOILA! Done.

In [27]:
#Write each passenger's data as its own sheet/tab in the output Excel file
from os import path

#Can't get it to save files outside of the working directory for some reason...
#filepath = path.expanduser(r"../../../Dropbox/'Shared Folder - Becky and Dave'/Finances/Commuting/")
filepath = dt.date.today().isoformat() + "_CommuterInvoices.xlsx"

writer = pd.ExcelWriter(filepath)

#Iterate through the unique passenger names in the dataframe
for name in full_record["Passenger Name"].unique():
    temp_df = full_record.groupby("Passenger Name").get_group(name)
    #Remove the Name field, as that will already be in the sheet name
    temp_df.to_excel(writer, sheet_name = name, index = False, columns = ["Date", "Commute Time"])

writer.save()

## Taking Care of the Final Bits

At this point, an Excel file should existig in the working directory (same directory as this notebook) that contains a sheet for each unique passenger name from the time period specified. At this stage, all that remains is to do the following:

1. Run the next block of code (which will show the final amount each rider owes in USD) 
3. Copy that Excel document into the Finances/Commuting folder (an alias exists in the working directory for ease of use)
2. Copy the data in each Excel sheet into an invoice Word doc for each passenger who owes money this month, using `CommuterInvoice_Template.docx` as the template for each invoice and using the naming schema `MM-YYYY_CommuterInvoice_PassengerName.docx`. 
    * Don't forget to put in the final amount they owe for that time period/month!


In [28]:
#How much does each rider owe??
full_record.groupby("Passenger Name").count()["Commute Time"] * GOING_RATE

Passenger Name
Courtney                        3
Paul                           18
Paul (credit - don't count)     6
Sarah                           3
Name: Commute Time, dtype: int64